# Combining running and music parameters
> Dislcaimer: This notebook is still only a stub.

In [3]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.pyplot as plt

import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [4]:
from mergait.utility import *
data_path = '../data/sample_full_obf/'

df_sessions = load_datadumps('sessions.csv', timestamp_columns=['t_start', 't_end'], base_path=data_path)

df_footpods, df_footpods_sc, df_music, df_phone_activity, df_phone_motion, df_phone_location = load_datadumps(['footpods.csv', 'footpods_sc.csv','music.csv','phone_activity.csv','phone_motion.csv.gz','phone_location.csv'], base_path=data_path)

df_music_features_tracks, df_music_features_sections = load_datadumps(['music_features_tracks.csv','music_features_sections.csv'], timestamp_columns=[], base_path=data_path)

df_sessions

,session_id,user_id,t_start,t_end,duration
0,s1,u0,1970-01-01 00:00:00.000000000,1970-01-01 01:02:26.141195059,3746.141195
1,s0,u0,1970-01-14 00:16:00.758486271,1970-01-14 01:22:31.301756143,3990.543270


In [ ]:
from mergait.filters import *

# filter out invalid sessions
df_sessions = apply_session_filters(df_sessions, df_footpods)
df_sessions

In [ ]:
from mergait.bouts import *

# filter invalid session data from the other dataframes
for df in [df_footpods, df_footpods_sc, df_music, df_phone_location, df_phone_activity, df_phone_motion]:
    log.debug("[ Initial length: {}".format(len(df)))
    add_bouts_as_column(
        df, df_sessions, new_column="session_id", valid_column="session_id"
    )
    df.dropna(subset=['session_id'], inplace=True)
    df.drop(['session_id'], axis=1, inplace=True)


In [5]:
from mergait.recipes import *

df_pod_steps, df_pod_symmetry = recipe_footpod_symmetry(df_footpods, df_music, df_phone_activity, df_sessions)

DEBUG:mergait-Recipes:[ Computing symmetry information from footpod data
DEBUG:mergait-Recipes:Computing aggregate statistics per song/section
DEBUG:mergait-Recipes:] Done, computed symmetry for 18063 cycles in 36 songs/sections


In [6]:
df_pod_steps_sections, df_pod_symmetry_sections = recipe_footpod_symmetry(df_footpods, df_music, df_phone_activity, df_sessions, sections=df_music_features_sections)

DEBUG:mergait-Recipes:[ Computing symmetry information from footpod data
DEBUG:mergait-Recipes:Computing aggregate statistics per song/section
DEBUG:mergait-Recipes:] Done, computed symmetry for 18063 cycles in 289 songs/sections


In [7]:
df_imu_steps, df_imu_symmetry = recipe_imu_symmetry(df_phone_motion, df_music, df_phone_activity, df_sessions)

DEBUG:mergait-Recipes:[ Computing symmetry information from imu vertical acceleration
DEBUG:MerGait:[ Extracting gait features from the vertical acceleration of a motion sensor
DEBUG:MerGait:Finding initial and final contact peaks
DEBUG:MerGait:Found 18815 IC and 18816 FC peaks
DEBUG:MerGait:Collecting contact pairs for feature extraction
DEBUG:MerGait:] Done, returning 15548 gait cycles
DEBUG:mergait-Recipes:Computing aggregate statistics per song/section
DEBUG:mergait-Recipes:[ Following recipe to compute Gait Symmetry Index per running bout
DEBUG:mergait-Recipes:] Computed the gsi for 166 bouts
DEBUG:mergait-Recipes:] Done, computed symmetry for 15048 cycles in 36 songs/sections


In [8]:
df_imu_steps_sections, df_imu_symmetry_sections = recipe_imu_symmetry(df_phone_motion, df_music, df_phone_activity, df_sessions, sections=df_music_features_sections)

DEBUG:mergait-Recipes:[ Computing symmetry information from imu vertical acceleration
DEBUG:MerGait:[ Extracting gait features from the vertical acceleration of a motion sensor
DEBUG:MerGait:Finding initial and final contact peaks
DEBUG:MerGait:Found 18815 IC and 18816 FC peaks
DEBUG:MerGait:Collecting contact pairs for feature extraction
DEBUG:MerGait:] Done, returning 15548 gait cycles
DEBUG:mergait-Recipes:Computing aggregate statistics per song/section
DEBUG:mergait-Recipes:[ Following recipe to compute Gait Symmetry Index per running bout
DEBUG:mergait-Recipes:] Computed the gsi for 400 bouts
DEBUG:mergait-Recipes:] Done, computed symmetry for 15048 cycles in 286 songs/sections
